In [1]:
import numpy as np
import awswrangler as wr

In [2]:
import pandas as pd, requests, boto3, pyarrow as pa, pyarrow.parquet as pq

In [3]:
s3 = boto3.resource('s3')

for bucket in s3.buckets.all():
    print(bucket.name)

anythingwild
ayodeji-data-ingestion-bucket
ayodeji-terraform-bucket
boto-awswrangler-demo
cde-athena-query-result
cde-aws-wrangler-demo
cde-blessed-madukoma-s3-bucket
cde-bukky-bucket
cde-capstone
cde-capstone-bucket-sam
cde-ebun-bucket
cde-hakeem-bucket
cde-hammed-adegunju
cde-leo-bucket
cde-pasky-bucket
cde-peace-bucket
cde-practice-travel-agency-bucket
cde-raw-data
cde-terraform-state
cde-test-tinubu
cde-your-s3-bucket-name
cdechiomabucket
chisomnwa-bucket
dami-cde-bucket
datawarehouse4me
dev-cde-travel-agency
dev-male-female-ingestion
dpe-custom-bucket
dpe-terraform-statefile
edsbucket7
femmyte-cde-bucket
greybuck1
leo-cde-project
leo-cde-project-backend
my-tf-female-bucket
olubusayo-bkt
olubusayo-blessing-friend
random-profile-terraform-state-bucket
random-user-extraction-demo
remote-state-bucket-vpc-stack
tbone-backend-bucket
travel-agency-data-lake
travel-agency-terraform-state-bucket
zainy-bucket


In [4]:
url = 'https://restcountries.com/v3.1/all'
r = requests.get(url)

In [5]:
df = pd.DataFrame(r.json())

In [6]:
df.columns

Index(['name', 'tld', 'cca2', 'ccn3', 'cca3', 'independent', 'status',
       'unMember', 'currencies', 'idd', 'capital', 'altSpellings', 'region',
       'languages', 'translations', 'latlng', 'landlocked', 'area', 'demonyms',
       'flag', 'maps', 'population', 'car', 'timezones', 'continents', 'flags',
       'coatOfArms', 'startOfWeek', 'capitalInfo', 'cioc', 'subregion', 'fifa',
       'borders', 'gini', 'postalCode'],
      dtype='object')

In [7]:
wr.s3.to_parquet(
    df=df.copy(),
    path="s3://dami-cde-bucket/raw/raw_data.parquet"
)


{'paths': ['s3://dami-cde-bucket/raw/raw_data.parquet'],
 'partitions_values': {}}

In [8]:
def extract_value(d, key,islist=False):
    try:
        value = d.get(key)
        if islist:
            return value[0]
        else:
            return value
    except:
        return None

In [58]:
df = wr.s3.read_parquet(
    path='s3://dami-cde-bucket/raw/raw_data.parquet'
)

In [59]:
def get_nested_values(d,return_type='val'):
    try:
        for key, value in d.items():
            if value is not None:
                if return_type == 'code':
                    return key
                elif return_type == 'name':
                    return value['name']
                elif return_type == 'symbol':
                    return value['symbol']
                else:
                    return value
        return None
    except:
        return None

In [60]:
#df['common_name'] = df['name'].apply(value_extract('common',col=df['name'])) 

# Key to extract

# Create a new column with extracted values
df['country_name'] = df['name'].apply(lambda d: extract_value(d, 'common'))
df['independence'] = df['independent']

df['united_nation_members'] = df['unMember']

df['startofweek'] = df['startOfWeek']

df['official_country_name'] = df['name'].apply(lambda d: extract_value(d, 'official'))

df['common_native_name'] = df['name'].apply(lambda d: extract_value(d, 'official'))

df['currency_code'] = df['currencies'].apply(lambda d: get_nested_values(d, 'code'))

df['currency_name'] = df['currencies'].apply(lambda d: get_nested_values(d, 'name'))

df['currency_symbol'] = df['currencies'].apply(lambda d: get_nested_values(d, 'symbol'))

df['country_code_root'] = df['idd'].apply(lambda d: extract_value(d, 'root'))

df['country_code_suffixes'] = df['idd'].apply(lambda d: extract_value(d, 'suffixes'))

df['country_code_suffixes'] = df.country_code_suffixes.apply(lambda x : x[0] if x is not None else x)

df['country_code_idd'] = df['country_code_root'] + df.country_code_suffixes 

df['capital'] = df.capital.apply(lambda x : x[0] if x is not None else x)

df['languages'] = df['languages'].apply(lambda d: get_nested_values(d))

df['continents'] = df.continents.apply(lambda x : x[0] if x is not None else x)


In [61]:
new_cols = [
    'country_name','independence','united_nation_members','startofweek','official_country_name','common_native_name','currency_code',
    'currency_name','currency_symbol','country_code_idd','capital','region','subregion','languages','continents'
    ]

df = df[new_cols]

In [62]:
df

,country_name,independence,united_nation_members,startofweek,official_country_name,common_native_name,currency_code,currency_name,currency_symbol,country_code_idd,capital,region,subregion,languages,continents
0,South Georgia,False,False,monday,South Georgia and the South Sandwich Islands,South Georgia and the South Sandwich Islands,SHP,Saint Helena pound,£,+500,King Edward Point,Antarctic,<NA>,English,Antarctica
1,Grenada,True,True,monday,Grenada,Grenada,XCD,Eastern Caribbean dollar,$,+1473,St. George's,Americas,Caribbean,English,North America
2,Switzerland,True,True,monday,Swiss Confederation,Swiss Confederation,CHF,Swiss franc,Fr.,+41,Bern,Europe,Western Europe,French,Europe
3,Sierra Leone,True,True,monday,Republic of Sierra Leone,Republic of Sierra Leone,SLL,Sierra Leonean leone,Le,+232,Freetown,Africa,Western Africa,English,Africa
4,Hungary,True,True,monday,Hungary,Hungary,HUF,Hungarian forint,Ft,+36,Budapest,Europe,Central Europe,Hungarian,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Belgium,True,True,monday,Kingdom of Belgium,Kingdom of Belgium,EUR,Euro,€,+32,Brussels,Europe,Western Europe,German,Europe
246,Israel,True,True,sunday,State of Israel,State of Israel,ILS,Israeli new shekel,₪,+972,Jerusalem,Asia,Western Asia,Arabic,Asia
247,New Zealand,True,True,monday,New Zealand,New Zealand,NZD,New Zealand dollar,$,+64,Wellington,Oceania,Australia and New Zealand,English,Oceania
248,Nicaragua,True,True,monday,Republic of Nicaragua,Republic of Nicaragua,NIO,Nicaraguan córdoba,C$,+505,Managua,Americas,Central America,Spanish,North America


In [ ]:
df.columns

Index(['name', 'tld', 'cca2', 'ccn3', 'cca3', 'independent', 'status',
       'unMember', 'currencies', 'idd', 'capital', 'altSpellings', 'region',
       'languages', 'translations', 'latlng', 'landlocked', 'area', 'demonyms',
       'flag', 'maps', 'population', 'car', 'timezones', 'continents', 'flags',
       'coatOfArms', 'startOfWeek', 'capitalInfo', 'cioc', 'subregion', 'fifa',
       'borders', 'gini', 'postalCode'],
      dtype='object')